In [4]:
import pandas as pd
import numpy as np
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
import os
os.getcwd()

'd:\\Users\\tommy.zhang\\Desktop\\DATS\\Singapore AI Lab\\AI Sharing\\1'

# Load Data

In [3]:
from os import listdir
from os.path import isfile, join

mypath = '/'
files = [f for f in listdir(mypath) if isfile(join(mypath, f)) if f.startswith('resale')]

df = pd.DataFrame()
for file in files:
    df_tmp = pd.read_csv(mypath+file)
    df = pd.concat([df,df_tmp], axis=0)

print(df.shape)
df.head()

(877412, 12)


,Station name,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,Jurong East,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bukit Batok,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bukit Gombak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Brickland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Choa Chu Kang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Get Lat  & Lon for street name

In [2]:
def getcoordinates(street):
    proxies = {'https': 'http://172.28.121.200:7070'}
    req = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+street+'&returnGeom=Y&getAddrDetails=Y&pageNum=1', verify=False, proxies=proxies)
    resultsdict = eval(req.text)
    if len(resultsdict['results'])>0:
        return (resultsdict['results'][0]['LATITUDE'], resultsdict['results'][0]['LONGITUDE'])
    else:
        pass

In [8]:
coordinateslist= []
count = 0
failed_count = 0
n = df['street_name'].nunique()
df_street_coor = pd.DataFrame({'street_name': df['street_name'].unique()})

for street in df['street_name'].unique():
    try:
        coord = getcoordinates(street)
        if len(coord)>0:
            count = count + 1
            print('Extracting',count,'out of',n,'streets')
            df_street_coor.loc[df_street_coor['street_name']==street, 'latitude'] = coord[0]
            df_street_coor.loc[df_street_coor['street_name']==street, 'longitude'] = coord[1]
    except:
        count = count + 1           
        failed_count = failed_count + 1
        print('Failed to extract',count,'out of',n,'streets')
print('Total Number of Addresses With No Coordinates',failed_count)

Extracting 1 out of 579 streets
Extracting 2 out of 579 streets
Extracting 3 out of 579 streets
Extracting 4 out of 579 streets
Extracting 5 out of 579 streets
Extracting 6 out of 579 streets
Extracting 7 out of 579 streets
Extracting 8 out of 579 streets
Extracting 9 out of 579 streets
Extracting 10 out of 579 streets
Extracting 11 out of 579 streets
Extracting 12 out of 579 streets
Extracting 13 out of 579 streets
Extracting 14 out of 579 streets
Extracting 15 out of 579 streets
Extracting 16 out of 579 streets
Extracting 17 out of 579 streets
Extracting 18 out of 579 streets
Extracting 19 out of 579 streets
Extracting 20 out of 579 streets
Extracting 21 out of 579 streets
Extracting 22 out of 579 streets
Extracting 23 out of 579 streets
Extracting 24 out of 579 streets
Extracting 25 out of 579 streets
Extracting 26 out of 579 streets
Extracting 27 out of 579 streets
Extracting 28 out of 579 streets
Extracting 29 out of 579 streets
Extracting 30 out of 579 streets
Extracting 31 out o

In [11]:
df_street_coor.to_csv('street_name_coord.csv', index=False)

# Get coordinates for MRTs

In [32]:
df_mrt = pd.read_csv('mrt.csv')
df_mrt = df_mrt.dropna()
df_mrt.head()

,Station name
0,Jurong East
1,Bukit Batok
2,Bukit Gombak
3,Brickland
4,Choa Chu Kang


In [33]:
coordinateslist= []
count = 0
failed_count = 0
n = df_mrt['Station name'].nunique()
df_station_coor = pd.DataFrame({'station_name': df_mrt['Station name'].unique()})

for station in df_mrt['Station name'].unique():
    try:
        coord = getcoordinates(station+ ' MRT STATION')
        if station=='Bayshore':
            print('Bayshore here!!!')
            print(coord)
        if len(coord)>0:
            count = count + 1
            print('Extracting',count,'out of',n,'stations')
            df_station_coor.loc[df_station_coor['station_name']==station, 'latitude'] = coord[0]
            df_station_coor.loc[df_station_coor['station_name']==station, 'longitude'] = coord[1]
    except:
        count = count + 1           
        failed_count = failed_count + 1
        print('Failed to extract',count,'out of',n,'stations')
print('Total Number of stations With No Coordinates',failed_count)
df_station_coor.to_csv('station_coord.csv', index=False)

Extracting 1 out of 189 stations
Extracting 2 out of 189 stations
Extracting 3 out of 189 stations
Failed to extract 4 out of 189 stations
Extracting 5 out of 189 stations
Extracting 6 out of 189 stations
Failed to extract 7 out of 189 stations
Extracting 8 out of 189 stations
Extracting 9 out of 189 stations
Extracting 10 out of 189 stations
Extracting 11 out of 189 stations
Extracting 12 out of 189 stations
Extracting 13 out of 189 stations
Extracting 14 out of 189 stations
Extracting 15 out of 189 stations
Extracting 16 out of 189 stations
Extracting 17 out of 189 stations
Extracting 18 out of 189 stations
Extracting 19 out of 189 stations
Extracting 20 out of 189 stations
Extracting 21 out of 189 stations
Extracting 22 out of 189 stations
Extracting 23 out of 189 stations
Extracting 24 out of 189 stations
Extracting 25 out of 189 stations
Extracting 26 out of 189 stations
Extracting 27 out of 189 stations
Extracting 28 out of 189 stations
Extracting 29 out of 189 stations
Extractin

# Original code

In [ ]:
import openpyxl
import numpy as np
def calculate_distance(x,y):
    return np.sqrt(((x[0]-y[0])*110.574)**2 + ((x[1]-y[1])*111.32)**2)
wb = openpyxl.load_workbook('Street_Name_List.xlsx')
wb2 = openpyxl.load_workbook('Shopping_Mall_List.xlsx')
sheet = wb['Sheet1']
sheet2 = wb2['Sheet1']
for row in range (2, sheet.max_row +1):
   Distance = 200
   MRT = 0
   if sheet['B'+str(row)].value is not None:
      for k in range (2, sheet2.max_row + 1):
         x1 = sheet['B'+str(row)].value
         x2 = float(sheet2['B'+str(k)].value)
         y1 = sheet['C'+str(row)].value
         y2 = float(sheet2['C'+str(k)].value)
         #print (type(x1),type(x2),y1,y2)
         Dis_Temp = calculate_distance([x1,y1],[x2,y2])
         if Dis_Temp < Distance:
            Distance = Dis_Temp
            MRT = k
         else:
            continue
   else:
      continue
   print(MRT)
   print(sheet2['A'+str(MRT)].value)
   sheet['F'+str(row)].value = sheet2['A'+str(MRT)].value
wb.save('Street_Name_List.xlsx')
print('Job Done.')